In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns
import pandas as pd
import gensim
import nltk
import re
import string
from curses.ascii import isprint

In [3]:
stop_words = {'a','about','above','across','after','again','against','all','almost','alone','along','already','also','although','always','among','an','and','another','any','anybody','anyone','anything','anywhere','are','area','areas','around','as','ask','asked','asking','asks','at','away','b','back','backed','backing','backs','be','became','because','become','becomes','been','before','began','behind','being','beings','best','better','between','big','both','but','by','c','came','can','cannot','case','cases','certain','certainly','clear','clearly','come','could','d','did','differ','different','differently','do','does','done','down','down','downed','downing','downs','during','e','each','early','either','end','ended','ending','ends','enough','even','evenly','ever','every','everybody','everyone','everything','everywhere','f','face','faces','fact','facts','far','felt','few','find','finds','first','for','four','from','full','fully','further','furthered','furthering','furthers','g','gave','general','generally','get','gets','give','given','gives','go','going','good','goods','got','great','greater','greatest','group','grouped','grouping','groups','h','had','has','have','having','he','her','here','herself','high','high','high','higher','highest','him','himself','his','how','however','i','if','important','in','interest','interested','interesting','interests','into','is','it','its','itself','j','just','k','keep','keeps','kind','knew','know','known','knows','l','large','largely','last','later','latest','least','less','let','lets','like','likely','long','longer','longest','m','made','make','making','man','many','may','me','member','members','men','might','more','most','mostly','mr','mrs','much','must','my','myself','n','necessary','need','needed','needing','needs','never','new','new','newer','newest','next','no','nobody','non','noone','not','nothing','now','nowhere','number','numbers','o','of','off','often','old','older','oldest','on','once','one','only','open','opened','opening','opens','or','order','ordered','ordering','orders','other','others','our','out','over','p','part','parted','parting','parts','per','perhaps','place','places','point','pointed','pointing','points','possible','present','presented','presenting','presents','problem','problems','put','puts','q','quite','r','rather','really','right','right','room','rooms','s','said','same','saw','say','says','second','seconds','see','seem','seemed','seeming','seems','sees','several','shall','she','should','show','showed','showing','shows','side','sides','since','small','smaller','smallest','so','some','somebody','someone','something','somewhere','state','states','still','still','such','sure','t','take','taken','than','that','the','their','them','then','there','therefore','these','they','thing','things','think','thinks','this','those','though','thought','thoughts','three','through','thus','to','today','together','too','took','toward','turn','turned','turning','turns','two','u','under','until','up','upon','us','use','used','uses','v','very','w','want','wanted','wanting','wants','was','way','ways','we','well','wells','went','were','what','when','where','whether','which','while','who','whole','whose','why','will','with','within','without','work','worked','working','works','would','x','y','year','years','yet','you','young','younger','youngest','your','yours','z'}

In [4]:
stop_words.add('re')
stop_words.add('RE')
stop_words.add('Re')
stop_words.add('fw')
stop_words.add('FW')
stop_words.add('Fw')
# stop_words.add('thx')
# stop_words.add('pls')
# stop_words.add('call')
# stop_words.add('pm')
# stop_words.add('am')

In [5]:
emails_table = pd.read_csv("data/emails/Emails.csv").dropna()
person_table = pd.read_csv("data/emails/Persons.csv").dropna()
alias_table = pd.read_csv("data/emails/Aliases.csv").dropna()
email_rec_table = pd.read_csv("data/emails/EmailReceivers.csv").dropna()

In [6]:
emails_table.head()

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
16,17,C05739573,THE YOUTH OF LIBYA,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739573...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J; Nuiand,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogirna, Ana A; Grantham, Chris W","Wednesday, September 12, 2012 9:57 PM",F-2015-04841,C05739573,05/13/2015,RELEASE IN FULL,Hope. See picture below Kamala sent.,"-ottgat,t\n4 44 4, 0\n.\n1.3.141.4443\nUNCLASS..."
17,18,C05739575,ONE MORE PHOTO,H,"Sherman, Wendy R",213.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739575...,F-2015-04841,...,"H; Mills, Cheryl D; Sullivan, Jacob J, Nuland,...","Sherman, Wendy R <ShermanWR@state.gov>","Escrogima, Ana A","Wednesday, September 12, 2012 10:03 PM",F-2015-04841,C05739575,05/13/2015,RELEASE IN FULL,Another photo.,c. 371.14\nUNCLASSIFIED\nU.S. Department of St...
65,66,C05739638,BRITS AND LIBYA,sullivanjj@state.gov,H,80.0,2011-04-02T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739638...,F-2015-04841,...,'sullivanjj@state.gov',H <hrod17@clintonemailcom>,Huma Abedin,"Saturday, April 02, 2011 1034 AM",F-2015-04841,C05739638,05/13/2015,RELEASE IN PART,"When is Chris Stevens going?\n---,",UNCLASSIFIED\nU.S. Department of State\nCase N...
82,83,C05739656,THE CABLE: ROMNEY TO COMPARE BENGHAZI TO ORIGI...,millscd@state.gov,H,80.0,2012-10-08T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH3/DOC_0C05739656...,F-2015-04841,...,"'miliscd@stategov""",H <hrod17@clintonemail.com>,"Reines, Philippe 1; Mills, Cheryl D; Adams, Da...","Monday, October 08, 2012 07:26 AM",F-2015-04841,C05739656,05/13/2015,RELEASE IN FULL,Can you call me?,UNCLASSIFIED\nU.S. Department of State\nCase N...


In [7]:
person_table.head()

,Id,Name
0,1,111th Congress
1,2,AGNA USEMB Kabul Afghanistan
2,3,AP
3,4,ASUNCION
4,5,Alec


In [8]:
alias_table.head()

,Id,Alias,PersonId
0,1,111th congress,1
1,2,agna usemb kabul afghanistan,2
2,3,ap,3
3,4,asuncion,4
4,5,alec,5


In [9]:
email_rec_table.head()

,Id,EmailId,PersonId
0,1,1,80
1,2,2,80
2,3,3,228
3,4,3,80
4,5,4,80


In [10]:
print len(emails_table)
print len(person_table)
print len(alias_table)
print len(email_rec_table)

908
513
850
9306


In [11]:
emails_table['doc'] = emails_table["ExtractedSubject"] + " " + emails_table["ExtractedBodyText"]

In [12]:
def clean_text(text):
    
    # Remove any beginning or ending whitespace
    text.strip()
    # Combine conjunctions
    text = re.sub(r"'", "", text)
    # Replace punctuation with space
    text = text.translate(string.maketrans("",""), string.punctuation)
    # Replace all control characters
    text = ''.join(char for char in text if isprint(char))
    # Remove Digits
    text = re.sub(r"[0-9]", " ", text)
    
    
    text = text.lower()
    
    text_list = [token for token in text.split() if token not in stop_words and len(token) > 1]
    
    return text_list

In [13]:
emails_table['token_list'] = emails_table['doc'].apply(lambda s: clean_text(s))

In [14]:
dictionary = gensim.corpora.Dictionary(list(emails_table['token_list']))

In [15]:
print dictionary.num_docs
print dictionary.num_pos
print dictionary.num_nnz

908
10321
9549


In [16]:
corpus = [dictionary.doc2bow(text) for text in emails_table['token_list']]

In [17]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [18]:
for topic in lda.print_topics(20):
    print topic
    print

(0, u'0.015*call + 0.012*pls + 0.008*talk + 0.008*mtg + 0.005*print + 0.005*conference + 0.005*meeting + 0.005*am + 0.005*hrc + 0.005*report')

(1, u'0.019*call + 0.011*time + 0.009*office + 0.008*list + 0.006*am + 0.006*dan + 0.006*ok + 0.006*speak + 0.006*pm + 0.005*care')

(2, u'0.023*call + 0.010*thx + 0.009*cheryl + 0.008*draft + 0.008*schedule + 0.007*pis + 0.006*friday + 0.006*tys + 0.006*calls + 0.006*send')

(3, u'0.021*clips + 0.020*press + 0.019*call + 0.019*doc + 0.017*dialogue + 0.016*strategic + 0.007*thx + 0.006*im + 0.006*help + 0.005*tomorrow')

(4, u'0.007*update + 0.006*tomorrow + 0.006*agree + 0.005*letter + 0.005*idea + 0.005*asap + 0.005*scheduled + 0.005*thats + 0.004*issue + 0.004*dc')

(5, u'0.020*call + 0.010*tomorrow + 0.008*list + 0.008*schedule + 0.007*ok + 0.006*pls + 0.006*send + 0.006*statement + 0.005*ideas + 0.005*close')

(6, u'0.020*schedule + 0.013*call + 0.010*im + 0.010*clips + 0.009*lona + 0.009*press + 0.008*hope + 0.008*strategic + 0.008*dialog

In [19]:
hdp = gensim.models.hdpmodel.HdpModel(corpus=corpus, id2word=dictionary)

In [20]:
for topic in hdp.print_topics():
    print topic
    print

topic 0: 0.003*youand + 0.002*lady + 0.002*carols + 0.002*elsewhere + 0.002*lanka + 0.002*proposed + 0.002*colombian + 0.002*renovate + 0.002*tomorrowcdm + 0.002*outsidehow + 0.002*valmoros + 0.001*weekoriginal + 0.001*collaborate + 0.001*additional + 0.001*changes + 0.001*soft + 0.001*waiver + 0.001*introduction + 0.001*conf + 0.001*alt

topic 1: 0.002*fr + 0.002*related + 0.002*representative + 0.002*emazils + 0.002*commdate + 0.002*representation + 0.002*spoke + 0.002*moved + 0.002*likethat + 0.002*butrather + 0.002*pm + 0.001*incredibly + 0.001*president + 0.001*am + 0.001*fsi + 0.001*anytime + 0.001*mosa + 0.001*staffed + 0.001*withus + 0.001*capacity

topic 2: 0.003*press + 0.003*doc + 0.003*clips + 0.003*strategic + 0.002*divoire + 0.002*dialogue + 0.002*march + 0.002*pelosi + 0.002*influence + 0.002*flexibility + 0.001*outstanding + 0.001*population + 0.001*effortsover + 0.001*yetor + 0.001*comingmonday + 0.001*molho + 0.001*played + 0.001*tech + 0.001*abedinalso + 0.001*prove
